# Master notebook

In [5]:
from __future__ import absolute_import, division, print_function 
import os
import re
import sys

sys.path.append(os.path.join('.', '..')) 
#import utils
import utils_DL
import utils_s160159 as u_s



import matplotlib.pyplot as plt
import numpy as np
# import sklearn.datasets
import tensorflow as tf

## Load data

In [8]:
data_dir = './../Data_1'
NUM_SUBJECTS = 2

In [13]:
#Load all subjects into memory
subjects_list = []
for i in range(1,NUM_SUBJECTS+1):
    print("Loading subject %d..." %(i))
    inputs_night1, targets_night1  = u_s.load_spectrograms(data_path=data_dir, subject_id=i, night_id=1)
    if(i!=20):
        inputs_night2, targets_night2  = u_s.load_spectrograms(data_path=data_dir, subject_id=i, night_id=2)
    else:
        inputs_night2 = np.empty((0,3,224,224),dtype='uint8')
        targets_night2 = np.empty((0,),dtype='uint8')           

    current_inputs = np.concatenate((inputs_night1,inputs_night2),axis=0)
    current_targets = np.concatenate((targets_night1, targets_night2),axis=0)

    mean_images = np.zeros((5,3,224,224))
    mean_images = np.zeros((5,224,224,3))
    mean_images[0,] = np.mean(current_inputs[current_targets==0,], axis=0)
    mean_images[1,] = np.mean(current_inputs[current_targets==1,], axis=0)
    mean_images[2,] = np.mean(current_inputs[current_targets==2,], axis=0)
    mean_images[3,] = np.mean(current_inputs[np.logical_or(current_targets==3, current_targets==4),], axis=0)
    mean_images[4,] = np.mean(current_inputs[current_targets==5,], axis=0)

    subjects_list.append([current_inputs,current_targets, mean_images])
    
    
# extract image shapes
IMAGE_SHAPE = subjects_list[0][0].shape

Loading subject 1...


/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Loading subject 2...


In [12]:
weights_dict = np.load(data_dir + '/vgg16_weights.npz', encoding='bytes')

## Building the model

In [66]:
# hyperameters
NUM_CLASSES = 6
HEIGTH, WIDTH, NCHANNELS = IMAGE_SHAPE[1], IMAGE_SHAPE[2], IMAGE_SHAPE[3]
PADDING = 'same'
L_RATE = 0.00001
TRAIN_FEATURES = False

In [67]:
# https://www.cs.toronto.edu/~frossard/vgg16/vgg16.py

# Load the weights into memory
weights_dict = np.load(data_dir +'vgg16_weights.npz', encoding='bytes')

def tf_conv2d(inputs, name, trainable=TRAIN_FEATURES):
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(shape=weights_dict[name + '_W'].shape, 
                                 initializer=tf.constant_initializer(weights_dict[name + '_W']), 
                                 name=scope + 'weights', 
                               trainable=trainable)
        
        conv = tf.nn.conv2d(inputs, kernel, strides=[1, 1, 1, 1], padding='SAME')
        
        biases = tf.get_variable(shape=weights_dict[name + '_b'].shape,
                                 initializer=tf.constant_initializer(weights_dict[name + '_b']), 
                                 trainable=trainable, name=scope + 'biases')

        return(tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope))
        

def tf_max_pooling2d(inputs, name):
    with tf.name_scope(name) as scope:
        return(tf.nn.max_pool(inputs,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name=scope))        

def tf_fully_con(inputs, name, shape=[None, 4096]):
    with tf.name_scope(name) as scope:    
        fc_W = tf.Variable(tf.truncated_normal(shape, dtype=tf.float32, stddev=1e-1), 
                            name=scope + 'weights', trainable=True)

        fc_b = tf.Variable(tf.constant(0.0, shape=[shape[1]], dtype=tf.float32),
                             trainable=True, name=scope + 'biases')
        
        in_flat = tf.reshape(inputs, [-1, shape[0]])
        return(tf.nn.relu(tf.nn.bias_add(tf.matmul(in_flat, fc_W), fc_b)))
        

In [68]:
# init model
tf.reset_default_graph()

# init placeholders
x_pl = tf.placeholder(tf.float32, [None, HEIGTH, WIDTH, NCHANNELS], name='input_placeholder')
y_pl = tf.placeholder(tf.float32, [None, NUM_CLASSES], name='target_placeholder')

with tf.variable_scope('VVG16_layer'):

    # level one
    conv1_1 = tf_conv2d(inputs=x_pl, name='conv1_1')
    conv1_2 = tf_conv2d(inputs=conv1_1, name='conv1_2')
    pool1 = tf_max_pooling2d(inputs=conv1_2, name='pool1')

    # level two
    conv2_1 = tf_conv2d(inputs=pool1, name='conv2_1')
    conv2_2 = tf_conv2d(inputs=conv2_1, name='conv2_2')
    pool2 = tf_max_pooling2d(inputs=conv2_2, name='pool2')

    # level three
    conv3_1 = tf_conv2d(inputs=pool2, name='conv3_1')
    conv3_2 = tf_conv2d(inputs=conv3_1, name='conv3_2')
    conv3_3 = tf_conv2d(inputs=conv3_2, name='conv3_3')
    pool3 = tf_max_pooling2d(inputs=conv3_3, name='pool_3')

    # level four
    conv4_1 = tf_conv2d(inputs=pool3, name='conv4_1')
    conv4_2 = tf_conv2d(inputs=conv4_1, name='conv4_2')
    conv4_3 = tf_conv2d(inputs=conv4_2, name='conv4_3')
    pool4 = tf_max_pooling2d(inputs=conv4_3, name='pool_4')

    # level five
    conv5_1 = tf_conv2d(inputs=pool4, name='conv5_1')
    conv5_2 = tf_conv2d(inputs=conv5_1, name='conv5_2')
    conv5_3 = tf_conv2d(inputs=conv5_2, name='conv5_3')
    pool5 = tf_max_pooling2d(inputs=conv5_3, name='pool_5')

    # level six
    fc6 = tf_fully_con(inputs=pool5, name='fc6', shape=[int(np.prod(pool5.get_shape()[1:])), 4096])
    fc6_dropout = tf.layers.dropout(inputs=fc6, name='fc6_dropout',rate=0.5)

    # level seven
    fc7 = tf_fully_con(inputs=fc6_dropout, name='fc7', shape=[4096, 4096])
    fc7_dropout = tf.layers.dropout(inputs=fc7, name='fc7_dropout',rate=0.5)

    # level eigth
    fc8 = tf_fully_con(inputs=fc7_dropout, name='fc7', shape=[4096, NUM_CLASSES])
    #fc8 = tf_fully_con(inputs=fc7_dropout, name='fc7', shape=[4096, 1000])
    
with tf.variable_scope('output_layer'):    
    l_out = tf.nn.softmax(fc8, name='l_out')

print('Model consits of ', utils_DL.num_params(), 'trainable parameters.')

Model consits of  119570438 trainable parameters.


In [69]:
## Launch TensorBoard, and visualize the TF graph
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.45)

#with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
#    tmp_def = utils_DL.rename_nodes(sess.graph_def, lambda s:"/".join(s.split('_',1)))
#    utils_DL.show_graph(tmp_def)

### OPTIMISATION

In [ ]:
with tf.variable_scope('loss'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_pl * tf.log(l_out+1e-8), reduction_indices=[1])

    # averaging over samples
    cross_entropy = tf.reduce_mean(cross_entropy)

with tf.variable_scope('training'):
    # defining our optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=L_RATE)

    # applying the gradients
    train_op = optimizer.minimize(cross_entropy)
    
    

    
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = tf.equal(tf.argmax(l_out, axis=1), tf.argmax(y_pl, axis=1))

    # averaging the one-hot encoded vector
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy = tf.reduce_mean(correct_prediction)

### Test flow for model

In [118]:
# Test the forward pass    
x_batch, y_batch = subjects_list[0][0][0:2], _

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    y_pred = sess.run(fetches=l_out, feed_dict={x_pl: x_batch})

assert y_pred.shape == np.zeros((len(x_batch),NUM_CLASSES)).shape, "ERROR the output shape is not as expected!" \
        + " Output shape should be " + str(l_out.shape) + ' but was ' + str(y_pred.shape)

print('Forward pass successful!')

Forward pass successful!


## Train the model

In [ ]:
### Training Loop
batch_size = 100
max_epochs = 10


valid_loss, valid_accuracy = [], []
train_loss, train_accuracy = [], []
test_loss, test_accuracy = [], []

# reset counter 
mnist_data.train._epochs_completed = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Begin training loop')

    try:
        while mnist_data.train.epochs_completed < max_epochs:
            _train_loss, _train_accuracy = [], []
            
            ## Run train op
            x_batch, y_batch = mnist_data.train.next_batch(batch_size)
            fetches_train = [train_op, cross_entropy, accuracy]
            feed_dict_train = {x_pl: x_batch, y_pl: y_batch}
            _, _loss, _acc = sess.run(fetches_train, feed_dict_train)
            
            _train_loss.append(_loss)
            _train_accuracy.append(_acc)
            

            ## Compute validation loss and accuracy
            if mnist_data.train.epochs_completed % 1 == 0 \
                    and mnist_data.train._index_in_epoch <= batch_size:
                train_loss.append(np.mean(_train_loss))
                train_accuracy.append(np.mean(_train_accuracy))

                fetches_valid = [cross_entropy, accuracy]
                
                feed_dict_valid = {x_pl: mnist_data.validation.images, y_pl: mnist_data.validation.labels}
                _loss, _acc = sess.run(fetches_valid, feed_dict_valid)
                
                valid_loss.append(_loss)
                valid_accuracy.append(_acc)
                print("Epoch {} : Train Loss {:6.3f}, Train acc {:6.3f},  Valid loss {:6.3f},  Valid acc {:6.3f}".format(
                    mnist_data.train.epochs_completed, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1]))
        
        
        test_epoch = mnist_data.test.epochs_completed
        while mnist_data.test.epochs_completed == test_epoch:
            x_batch, y_batch = mnist_data.test.next_batch(batch_size)
            feed_dict_test = {x_pl: x_batch, y_pl: y_batch}
            _loss, _acc = sess.run(fetches_valid, feed_dict_test)
            test_loss.append(_loss)
            test_accuracy.append(_acc)
        print('Test Loss {:6.3f}, Test acc {:6.3f}'.format(
                    np.mean(test_loss), np.mean(test_accuracy)))


    except KeyboardInterrupt:
        pass
    
epoch = np.arange(len(train_loss))
plt.figure()
plt.plot(epoch, train_accuracy,'r', epoch, valid_accuracy,'b')
plt.legend(['Train Acc','Validation Acc'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim([0.75,1.03])
plt.show()